# Session 2-- Manipulating data and regression 


## Session 2 is composed of 3 parts¶

1. Module 3: Manipulating data
    - Notes (Current page)
    - Video
2. Module 4: Regression
    - Notes
    - Video

3. Excercise to practice material covered in Modules 3 & 4
    - Link



# Module 4